In [1]:
!pip install --upgrade ultralytics pyyaml

In [2]:
import os
import subprocess # To run shell commands
import yaml
from pathlib import Path
from ultralytics import YOLO

In [6]:
download_url = "https://universe.roboflow.com/ds/k9p2XS5rLi?key=xNqlMAzW4M"
zip_file = "roboflow.zip"
data_yaml_path = None # Initialize path variable

# --- Execute Download Command ---
print(f"Downloading dataset from Roboflow using curl...")
curl_command = ["curl", "-L", download_url, "-o", zip_file]
try:
    result = subprocess.run(curl_command, check=True, capture_output=True, text=True)
    print("Download successful.")
    # print("Curl output (stderr):", result.stderr) # Uncomment for detailed curl progress/errors
except subprocess.CalledProcessError as e:
    print(f"Error during curl download: {e}")
    print(f"Stderr: {e.stderr}")
except FileNotFoundError:
    print("Error: 'curl' command not found. Is it installed and in your WSL PATH?")
except Exception as e:
    print(f"An unexpected error occurred during download: {e}")

Download successful.


In [7]:
# --- Execute Unzip Command ---
if Path(zip_file).exists():
    print(f"Unzipping {zip_file}...")
    unzip_command = ["unzip", "-q", zip_file] # -q for quiet unzip
    try:
        # Run unzip in the current directory
        result = subprocess.run(unzip_command, check=True, capture_output=True, text=True)
        print("Unzip successful.")

        # --- Locate data.yaml ---
        # After unzipping, find the data.yaml file. It might be in the current dir or a subdir.
        print("Searching for data.yaml...")
        possible_paths = list(Path.cwd().glob('**/data.yaml')) # Search current dir and subdirs

        if not possible_paths:
             print("ERROR: data.yaml not found after unzip.")
             print("Please check the contents of the unzipped folder.")
        elif len(possible_paths) > 1:
             print(f"Warning: Found multiple data.yaml files: {possible_paths}")
             data_yaml_path = possible_paths[0] # Use the first one found
             print(f"Using: {data_yaml_path}")
        else:
             data_yaml_path = possible_paths[0]
             print(f"Found data.yaml at: {data_yaml_path.resolve()}")
             # Optional: Print content
             # with open(data_yaml_path, 'r') as f:
             #      print("\nContent of data.yaml:")
             #      print(f.read())

    except subprocess.CalledProcessError as e:
        print(f"Error during unzip: {e}")
        print(f"Stderr: {e.stderr}")
    except FileNotFoundError:
        print("Error: 'unzip' command not found. Is it installed and in your WSL PATH?")
    except Exception as e:
        print(f"An unexpected error occurred during unzip/search: {e}")

    # --- Execute Remove Zip Command ---
    print(f"Removing {zip_file}...")
    rm_command = ["rm", zip_file]
    try:
        result = subprocess.run(rm_command, check=True)
        print(f"{zip_file} removed.")
    except subprocess.CalledProcessError as e:
        print(f"Error removing {zip_file}: {e}")
    except FileNotFoundError:
         print("Error: 'rm' command not found.")
    except Exception as e:
        print(f"An unexpected error occurred removing zip: {e}")

else:
    print("Skipping unzip and cleanup because download failed or zip file doesn't exist.")


Unzipping roboflow.zip...
Unzip successful.
Searching for data.yaml...
Found data.yaml at: /home/user/Cortana/machine-learning/data.yaml
Removing roboflow.zip...
roboflow.zip removed.


In [8]:
if data_yaml_path and data_yaml_path.exists():
    # Choose a model supported by the ultralytics library
    model_choice = 'yolov12n.pt' 

    print(f"\nLoading model: {model_choice}")
    model = YOLO(model_choice)

    print(f"Starting training using dataset config: {data_yaml_path.resolve()}")

    try:
        model.train(
            data=str(data_yaml_path.resolve()), # Use the located data.yaml path
            epochs=50,
            imgsz=640,
            batch=16,
            lr0=0.001,
            patience=10,
            augment=True,
            name='curl-streetview-yolov8n' # Updated experiment name
            # Add device argument if needed: device=0 for GPU, device='cpu', device='mps'
        )
        print("\nTraining finished.")
        # Results are saved in runs/detect/curl-streetview-yolov8n*
    except Exception as e:
        print(f"\nAn error occurred during training: {e}")
        print("Ensure your environment (CUDA, PyTorch, GPU drivers) is correctly set up if using GPU.")
        print(f"Check that the model '{model_choice}' is supported or available.")

else:
    print("\nSkipping training because the data.yaml path was not found or set correctly.")


Loading model: yolov12n.pt
Starting training using dataset config: /home/user/Cortana/machine-learning/data.yaml
Ultralytics 8.3.118 🚀 Python-3.12.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX A6000, 48566MiB)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=/home/user/Cortana/machine-learning/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=curl-streetview-yolov8n, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False

100%|██████████| 5.35M/5.35M [00:00<00:00, 62.5MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2550.6±550.7 MB/s, size: 97.8 KB)


train: Scanning /home/user/Cortana/machine-learning/train/labels... 5805 images, 93 backgrounds, 0 corrupt: 100%|██████████| 5805/5805 [00:05<00:00, 1038.81it/s]


train: New cache created: /home/user/Cortana/machine-learning/train/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1415.1±730.6 MB/s, size: 85.6 KB)


...

      50/50      5.44G     0.7609     0.4113     0.8767        146        640: 100%|██████████| 363/363 [00:45<00:00,  7.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.66it/s]

                   all        549       6270      0.904      0.893      0.931      0.719



50 epochs completed in 0.672 hours.
Optimizer stripped from runs/detect/curl-streetview-yolov8n/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/curl-streetview-yolov8n/weights/best.pt, 5.5MB

Validating runs/detect/curl-streetview-yolov8n/weights/best.pt...
Ultralytics 8.3.118 🚀 Python-3.12.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX A6000, 48566MiB)
YOLOv12n summary (fused): 159 layers, 2,557,703 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:08<00:00,  2.02it/s]


                   all        549       6270      0.885      0.882      0.927      0.709
               bicycle        189        250      0.878      0.924      0.936      0.764
                   bus         81        108       0.92      0.935      0.972      0.876
                   car        520       3842      0.909      0.938      0.963       0.77
             motorbike        331       1238      0.892      0.817      0.893      0.567
                person        196        832      0.826      0.797       0.87       0.57
Speed: 0.1ms preprocess, 3.7ms inference, 0.0ms loss, 6.7ms postprocess per image
Results saved to runs/detect/curl-streetview-yolov8n

Training finished.
